In [3]:
import pandas as pd
import requests
import os
import fitz
import re
import threading
import time

In [4]:
df = pd.read_excel("D:\DS\dibimbing\project python oop\day 2 python\multithreading_assignment\BATAM.xlsx")

In [5]:
df.head(5)

,title,s3_path
0,Peraturan Wali Kota Batam Nomor 23 Tahun 2022,http://jdih.batam.go.id/dokumen-hukum/storage/...
1,Peraturan Wali Kota Batam Nomor 53 Tahun 2023,http://jdih.batam.go.id/dokumen-hukum/storage/...
2,Kesepakatan bersama antara pemerintah kota bat...,http://jdih.batam.go.id/dokumen-hukum/storage/...
3,Peraturan Wali Kota Batam Nomor 8 Tahun 2023,http://jdih.batam.go.id/dokumen-hukum/storage/...
4,Peraturan Walikota Batam Nomor 1 Tahun 2022,http://jdih.batam.go.id/dokumen-hukum/storage/...


In [6]:
#melakukan cleansing terhadap kolom title dan s3_path untuk menghapus spasi di awal dan di akhir
df['title'] = df['title'].str.strip()
df['s3_path'] = df['s3_path'].str.strip()

In [7]:
# cek apakah folder file_pdf sudah ada atau belum, jika belum maka buat folder file_pdf
if not os.path.exists('file_pdf'):
    os.makedirs('file_pdf')

# membuat fungsi untuk mendownload file pdf dengan library request dan link yang ada di file excel
def download_file(url, filename=None):
    response = requests.get(url)
    if not filename:
        filename = os.path.basename(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

# melakukan looping terhadap setiap baris data dan melakukan download melalui link yang ada di kolom s3_path
for index, row in df.iterrows():
    title = row['title']
    s3_path = row['s3_path']

    # mengambil ekstensi file setelah karakter . (pdf)
    file_extension = s3_path.split('.')[-1]

    # membuat nama file berdasarkan URL
    base_filename = os.path.basename(s3_path)

    # melakukan cleansing terhadap kolom title dan menggabungkannya dengan ekstensi file yang sudah diekstrak serta dibatasi titlenya menjadi 100 karakter
    clean_title = re.sub(r'[^\w\s]', ' ', title)[:100]

    filename = f"file_pdf/{clean_title}.{file_extension}"

    # melakukan download terhadap file
    try:
        download_file(s3_path, filename)
        print(f"File {title} downloaded successfully.")
    except Exception as e:
        print(f"Error downloading {title}: {e}")

File Peraturan Wali Kota Batam Nomor 23 Tahun 2022 downloaded successfully.
File Peraturan Wali Kota Batam Nomor 53 Tahun 2023 downloaded successfully.
File Kesepakatan bersama antara pemerintah kota batam dan pt bank pembangunan daerah jawa barat dan banten, Tbk cabang batam downloaded successfully.
File Peraturan Wali Kota Batam Nomor 8 Tahun 2023 downloaded successfully.
File Peraturan Walikota Batam Nomor 1 Tahun 2022 downloaded successfully.
File Peraturan Wali Kota Batam Nomor 17 Tahun 2023 downloaded successfully.
File Peraturan Daerah Kota Batam Nomor 7 Tahun 2022 downloaded successfully.
File Peraturan Wali Kota Batam Nomor 18 Tahun 2022 downloaded successfully.
File Peraturan Wali Kota Batam Nomor 51 Tahun 2023 downloaded successfully.
File Kesepakatan bersama antara pemerintah kota batam dan politeknik negeri batam downloaded successfully.
File Peraturan Wali Kota Batam Nomor 55 Tahun 2022 downloaded successfully.
File Peraturan Wali Kota Batam Nomor 13 Tahun 2022 downloaded

In [8]:
# Fungsi untuk mencari kata kunci dalam file PDF
def find_keywords_in_pdf(pdf_file_path, keywords):
    results = {keyword: [] for keyword in keywords}
    try:
        doc = fitz.open(pdf_file_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            page_text = page.get_text()
            for keyword in keywords:
                if keyword in page_text:
                    results[keyword].append(os.path.basename(pdf_file_path))
        doc.close()
    except Exception as e:
        print(f"Error opening {pdf_file_path}: {e}")
    return results

# Path ke folder file_pdf
pdf_folder = 'file_pdf'

# Kata kunci yang ingin dicari
keywords = ["sanksi", "berlaku sampai dengan"]

# Dictionary untuk menyimpan hasil pencarian
search_results = {keyword: [] for keyword in keywords}

# Melakukan looping untuk setiap file PDF dalam folder file_pdf
start_time = time.time()  # Record the starting time

for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_folder, filename)
        # Mencari kata kunci dalam file PDF
        found_keywords = find_keywords_in_pdf(file_path, keywords)
        for keyword in keywords:
            search_results[keyword].extend(found_keywords[keyword])

end_time = time.time()  # Record the ending time

# Menampilkan hasil pencarian
print("kata                   | list peraturan")
print("-----------------------------------------")
for keyword in keywords:
    files_found = ", ".join(set(search_results[keyword]))
    print(f"{keyword.ljust(22)} | {files_found}")

process_time = end_time - start_time
print(f"Process time: {process_time:.2f} seconds")

Error opening file_pdf/Peraturan Walikota Batam Nomor 5 Tahun 2005.pdf: cannot open broken document
Error opening file_pdf/Peraturan Daerah Nomor 3 Tahun 2022.pdf: cannot open broken document
Error opening file_pdf/Peraturan Daerah Kota Batam Nomor 3 Tahun 2016.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 12 Tahun 2011.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 13 Tahun 2021.pdf: cannot open broken document
Error opening file_pdf/Surat Edaran Walikota Batam Nomor 61 Tahun 2021.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 6 Tahun 2001.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 36 Tahun 2010.pdf: cannot open broken document
Error opening file_pdf/KESEPAKATAN BERSAMA ANTARA PEMERINTAH KABUPATEN BOGOR PROVINSI JAWA BARAT DENGAN PEMERINTAH KOTA BAT.pdf: cannot open broken document
Error opening file_pdf/Peraturan Daerah K

In [11]:
# Fungsi untuk mencari kata kunci dalam file PDF
def find_keywords_in_pdf(pdf_file_path, keywords, results):
    try:
        doc = fitz.open(pdf_file_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            page_text = page.get_text()
            for keyword in keywords:
                if keyword in page_text:
                    results[keyword].append(os.path.basename(pdf_file_path))
        doc.close()
    except Exception as e:
        print(f"Error opening {pdf_file_path}: {e}")

# Path ke folder file_pdf
pdf_folder = 'file_pdf'

# Kata kunci yang ingin dicari
keywords = ["sanksi", "berlaku sampai dengan"]

# Dictionary untuk menyimpan hasil pencarian
search_results = {keyword: [] for keyword in keywords}

# Melakukan looping untuk setiap file PDF dalam folder file_pdf
def process_pdf_files(files):
    for file_path in files:
        find_keywords_in_pdf(file_path, keywords, search_results)

# List file PDF dalam folder file_pdf
pdf_files = [os.path.join(pdf_folder, filename) for filename in os.listdir(pdf_folder) if filename.endswith('.pdf')]

# Jumlah thread yang akan digunakan
num_threads = 8

# Membagi file PDF ke dalam beberapa bagian untuk setiap thread
pdf_files_per_thread = [pdf_files[i:i + len(pdf_files) // num_threads] for i in range(0, len(pdf_files), len(pdf_files) // num_threads)]

# Membuat dan menjalankan thread
threads = []
start_time = time.time()  # Record the starting time

for i in range(num_threads):
    thread = threading.Thread(target=process_pdf_files, args=(pdf_files_per_thread[i],))
    thread.start()
    threads.append(thread)

# Menunggu semua thread selesai
for thread in threads:
    thread.join()

end_time = time.time()  # Record the ending time

# Menampilkan hasil pencarian
print("kata                   | list peraturan")
print("-----------------------------------------")
for keyword in keywords:
    files_found = ", ".join(set(search_results[keyword]))
    print(f"{keyword.ljust(22)} | {files_found}")

process_time = end_time - start_time
print(f"Process time: {process_time:.2f} seconds")

Error opening file_pdf/Peraturan Walikota Batam Nomor 1 Tahun 2005.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 5 Tahun 2005.pdf: cannot open broken document
Error opening file_pdf/Peraturan Daerah Nomor 3 Tahun 2022.pdf: cannot open broken document
Error opening file_pdf/Peraturan Daerah Kota Batam Nomor 3 Tahun 2016.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 1 Tahun 2004.pdf: cannot open broken documentError opening file_pdf/Putusan Pengadilan Negeri Batam Nomor 25 PDT G 2017 PN BTM.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 13 Tahun 2011.pdf: cannot open broken document

Error opening file_pdf/Peraturan Daerah Kota Batam Nomor 5 Tahun 2014.pdf: cannot open broken document
Error opening file_pdf/Peraturan Daerah Kota Batam Nomor 4 Tahun 2014.pdf: cannot open broken document
Error opening file_pdf/Peraturan Walikota Batam Nomor 16 Tahun 2002.pdf: cannot ope

kesimpulan : multi threading berpengaruh terhadap kecepatan pemrosesan. Dengan menggunakan single threading, waktu eksekusi adalah 34 detik, sedangkan dengan multi threading, waktu eksekusi menjadi lebih cepat, yaitu hanya 30 detik. Dengan kata lain, multi threading dapat menyelesaikan tugas lebih cepat dibandingkan dengan single threading.

Penggunaan multi threading juga memungkinkan program untuk menjalankan beberapa tugas secara bersamaan (concurrent) dengan membagi pekerjaan menjadi beberapa thread yang bekerja secara paralel. Sementara pada single threading, tugas dijalankan satu per satu secara berurutan.